__<div align='center'><font size='10'>BOSON FACS</font></div>__

In [1]:
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
# import statsmodels.api as sm
import csv
# import pycircos
from scipy.stats import poisson
import itertools
import collections
import seaborn as sns

import FACSus as fs

# sns.set(style="whitegrid")
%cd /gpfs3/well/ansari/users/gjx698/BOSON_FACS
# %cd ..

/gpfs3/well/ansari/users/gjx698/BOSON_FACS


In [2]:
df1 = pd.read_csv('facs/Baseline/Panel1_Baseline.csv', sep = ',', dtype = {'Patient ID': 'str'})
df2 = pd.read_csv('facs/Baseline/Panel2_Baseline.csv', sep = ',', dtype = {'Patient ID': 'str'})
df3 = pd.read_csv('facs/Baseline/Panel3_Baseline.csv', sep = ',', dtype = {'Patient ID': 'str'})
df4 = pd.read_csv('facs/Baseline/Panel4_Baseline.csv', sep = ',', dtype = {'Patient ID': 'str'})
df5 = pd.read_csv('facs/Baseline/Panel5_Baseline.csv', sep = ',', dtype = {'Patient ID': 'str'})
df1.iloc[:,1:-9] = df1.iloc[:,1:-9].apply(pd.to_numeric)
df1 = df1.drop(columns = ['Azim data.SUBJID'])
df2.iloc[:, 1:] = df2.iloc[:, 1:].apply(pd.to_numeric)
df3.iloc[:, 1:] = df3.iloc[:, 1:].apply(pd.to_numeric)
df4.iloc[:, 1:] = df4.iloc[:, 1:].apply(pd.to_numeric)
df5.iloc[:, 1:] = df5.iloc[:, 1:].apply(pd.to_numeric)

dfs = [df1, df2, df3, df4, df5]
new_dfs = [df1.copy(), df2.copy(), df3.copy(), df4.copy(), df5.copy()]
for i in range(5):
    new_columns = [dfs[i].columns[0]] + [col + '_panel' + str(i+1) for col in dfs[i].columns[1:]]
    new_dfs[i].columns = new_columns
tmp = pd.merge(new_dfs[0], new_dfs[1], on = 'Patient ID')
tmp = pd.merge(tmp, new_dfs[2], on = 'Patient ID')
tmp = pd.merge(tmp, new_dfs[3], on = 'Patient ID')
df = pd.merge(tmp, new_dfs[4], on = 'Patient ID')
df = df.dropna()
# df['Patient ID'] = 'BOSON' + df['Patient ID']